In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

from xgboost import XGBClassifier
#from lightgbm import LGBMClassifier

In [2]:
pwd

'C:\\Users\\635718\\Desktop\\DataScience\\HackerEarth\\MLC4'

In [3]:
train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')

In [4]:
id_test = test['connection_id'].values

In [ ]:
train.shape

In [ ]:
train.columns

In [ ]:
train.nunique()

In [ ]:
train.head()

In [ ]:
train[['cont_11','cont_12','cont_13','cont_17']]

In [ ]:
sns.distplot(train['cont_3'])

In [ ]:
train['target'].value_counts()

In [ ]:
sns.distplot(train['target'], kde=False)

In [ ]:
test.head()

In [ ]:
test.nunique()

In [ ]:
sns.jointplot(x='cat_9',y='cat_22',data=train)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=100, max_depth=3, min_samples_leaf=4, max_features=0.2, random_state=0)
gb.fit(train.drop(['connection_id', 'target'],axis=1), train.target)
features = train.drop(['connection_id', 'target'],axis=1).columns.values
print("----- Training Done -----")

In [ ]:
x, y = (list(x) for x in zip(*sorted(zip(gb.feature_importances_, features), 
                                                            reverse = False)))

In [ ]:
print(gb.feature_importances_)
print(features)

In [ ]:
feature_imp = pd.DataFrame(features)
feature_imp.columns = ['feature']
feature_imp['imp'] = gb.feature_importances_

In [ ]:
feature_imp.sort_values(by='imp',ascending=False)

In [ ]:
import matplotlib.pyplot as plt

#correlation matrix
corrmat = train.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);

In [ ]:
#sns.jointplot(x='feature',y='imp',data=feature_imp)

In [ ]:
features = ['cat_20','cont_3','cat_2','cont_2','cat_22','cont_14','cont_12','cat_23','cat_3','cont_11','cat_9',
            'cont_13','cat_21','cat_7','cat_1','cont_1','cont_17','cont_8','cont_9','cont_18','cat_8','cont_10',
            'cont_15','cat_5','cat_10','cont_16','cont_5','cont_4','cont_7','cont_6','cat_19','cat_11']

In [ ]:
train_features = [x for x in train.columns if x not in ['connection_id','target']]

X = train[train_features].values
y = train.loc[:,'target'].values
T = test[train_features].values

#create the models
# LightGBM params
lgb_params_1 = {
    'learning_rate': 0.01,
    'n_estimators': 1250,
    'max_bin': 10,
    'subsample': 0.8,
    'subsample_freq': 10,
    'colsample_bytree': 0.8, 
    'min_child_samples': 500
}

lgb_params_2 = {
    'learning_rate': 0.005,
    'n_estimators': 3700,
    'subsample': 0.7,
    'subsample_freq': 2,
    'colsample_bytree': 0.3,  
    'num_leaves': 16
}

lgb_params_3 = {
   'objective':'binary:logistic',
   'learning_rate':0.02,
    'n_estimators':1000,
    'max_depth':4,
    'subsample':0.9,
    'colsample_bytree':0.9,  
    'min_child_weight':10
}

lgb_model_1 = LGBMClassifier(**lgb_params_1)
lgb_model_2 = LGBMClassifier(**lgb_params_2)
lgb_model_3 = XGBClassifier(**lgb_params_3)
#base_models = (lgb_model_1, lgb_model_2, lgb_model_3)
base_models = (lgb_model_1, lgb_model_2)

log_model = LogisticRegression()
stacker = log_model
print("models created")

In [9]:
#train_features = features
train_features = [x for x in train.columns if x not in ['connection_id','target']]

X = train[train_features].values
y = train.loc[:,'target'].values
T = test[train_features].values

param_1 = {"n_estimators": 400,
            "max_depth": 1,
            "min_samples_split": 0.1,
            "min_samples_leaf": 1,
            "max_leaf_nodes": 2,
            "min_weight_fraction_leaf": 0.2
}

param_2 = {"n_estimators": 800,
            "max_depth": 5,
            "min_samples_split": 0.4,
            "min_samples_leaf": 4,
            "max_leaf_nodes": 10,
            "min_weight_fraction_leaf": 0.3
}

param_3 = {"n_estimators": 300,
            "max_depth": 7,
            "min_samples_split": 0.5,
            "min_samples_leaf": 10,
            "max_leaf_nodes": 7,
            "min_weight_fraction_leaf": 0.4
}

#clf1 = RandomForestClassifier(**param_1)
#clf2 = RandomForestClassifier(**param_2)
#clf3 = RandomForestClassifier(**param_3)

lgb_params_3 = {
   'learning_rate':0.02,
    'n_estimators':100,
    'max_depth':4,
    'subsample':0.9,
    'colsample_bytree':0.9,  
    'min_child_weight':10
}

clf1 = XGBClassifier(**lgb_params_3)
clf2 = XGBClassifier(**lgb_params_3)
clf3 = XGBClassifier(**lgb_params_3)

base_models = (clf1, clf2, clf3)

print("models created")

models created


In [11]:
#now we have the data with equal set of positives and negatives
#lets check cross validation scores
from scipy.stats import mode
n_splits=3
folds = list(StratifiedKFold(n_splits, shuffle=True, random_state=10).split(X, y))
S_train = np.zeros((X.shape[0], len(base_models)))
S_test = np.zeros((T.shape[0], len(base_models)))

for i, clf in enumerate(base_models):
    S_test_i = np.zeros((T.shape[0], n_splits))
    for j, (train_idx, test_idx) in enumerate(folds):
        X_train = X[train_idx]
        y_train = y[train_idx]
        X_holdout = X[test_idx]
        print(X_train.shape)
        
        # Get positive examples
        """pos_1 = pd.Series(y_train == 1)
        pos_2 = pd.Series(y_train == 2)
        # Add positive examples - 1
        X_train = pd.concat([pd.DataFrame(X_train), pd.DataFrame(X_train[pos_1])])
        y_train = pd.concat([pd.DataFrame(y_train), pd.DataFrame(y_train[pos_1])])
        # Add positive examples - 2
        X_train = pd.concat([pd.DataFrame(X_train), pd.DataFrame(X_train[pos_2])])
        y_train = pd.concat([pd.DataFrame(y_train), pd.DataFrame(y_train[pos_2])])
        
        # Shuffle data
        idx = np.arange(len(X_train))
        np.random.shuffle(idx)
        X_train = X_train.iloc[idx]
        y_train = y_train.iloc[idx]"""

        print ("Fit %s fold %d" % (str(clf).split('(')[0], j+1))
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_holdout)
        #y_pred = clf.predict_proba(X_holdout)[:,1]  
        
        S_train[test_idx, i] = y_pred
        S_test_i[:, j] = clf.predict(T)
        #S_test_i[:, j] = clf.predict_proba(T)[:,1]
    
    #print(S_test_i)
    #print(mode(S_test_i, axis=1)[0].flatten())
    #print(S_test_i.mean(axis=1))
    S_test[:, i] = mode(S_test_i, axis=1)[0].flatten()
    #S_test[:, i] = S_test_i.mean(axis=1)
    #sns.distplot(S_test[:, i],kde=False)

(112871, 41)
Fit XGBClassifier fold 1
(112871, 41)
Fit XGBClassifier fold 2
(112872, 41)
Fit XGBClassifier fold 3
(112871, 41)
Fit XGBClassifier fold 1
(112871, 41)
Fit XGBClassifier fold 2
(112872, 41)
Fit XGBClassifier fold 3
(112871, 41)
Fit XGBClassifier fold 1
(112871, 41)
Fit XGBClassifier fold 2
(112872, 41)
Fit XGBClassifier fold 3


In [ ]:
from sklearn.grid_search import GridSearchCV
clf = RandomForestClassifier()
param_grid = {"n_estimators": [100,200,300],
              "max_depth": [1,5,10],
              "min_samples_split": [0.1,0.3,0.5,0.7],
              "min_samples_leaf": [1,5,7,9],
              "max_leaf_nodes": [2,5,7,9],
              "min_weight_fraction_leaf": [0.1,0.2,0.3,0.4]}

grid_search = GridSearchCV(clf, param_grid=param_grid, cv=10)
grid_search.fit(X, y)
print(grid_search.best_params_)

In [12]:
"""rf_params = {
    'n_estimators':500,
    'max_depth':4,
    "min_samples_split": 6,
    "min_samples_leaf": 10,
    "max_leaf_nodes": 10,
    "min_weight_fraction_leaf": 0.3
}"""

stacker = RandomForestClassifier()

results = cross_val_score(stacker, S_train, y, cv=5)
print("Stacker score: %.5f" % (results.mean()))
print(results)

#print(stacker.best_params_)

#print("S train size is : ", S_train.shape)
stacker.fit(S_train, y)
res = stacker.predict(S_test)

print(res)

Stacker score: 0.77877
[ 0.77942827  0.77836513  0.77857185  0.77918549  0.77829297]
[1 0 0 ..., 0 0 1]


In [ ]:
print(len(res))
print(len(id_test))

In [ ]:
sns.distplot(res,kde=False)

In [ ]:
print(results)
print(res)

sub = pd.DataFrame()
sub['id'] = id_test
sub['target'] = res
sub.to_csv('stacked_result_strat_upsample.csv', index=False)

print('completed')